In [8]:
import tempfile
import os



import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.estimator

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import FeatureSpace

from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc



ImportError: cannot import name 'FeatureSpace' from 'tensorflow.keras.utils' (/usr/local/lib/python3.10/dist-packages/keras/api/_v2/keras/utils/__init__.py)

In [3]:
dftrain = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
dfeval = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/eval.csv")
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']

NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

In [4]:
print(dftrain["embark_town"].unique())

['Southampton' 'Cherbourg' 'Queenstown' 'unknown']


In [5]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function(): #inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df)) #create tf.data.Dataset object with data and its la
    if shuffle:
      ds = ds.shuffle(1000) #randemize order data
    ds = ds.batch(batch_size).repeat(num_epochs) #split dataset into batches of 32 repeat process for number of epochs

    return ds #return a batch of datasets

  return input_function #return a function object for use


train_input_fn = make_input_fn(dftrain, y_train) #here we will call the input_function that was returned to us to get a data
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result['accuracy'])



0.7613636


In [23]:
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[10])
print(y_eval.loc[10])
print(result[10]['probabilities'][1])

sex                       female
age                         28.0
n_siblings_spouses             0
parch                          0
fare                        7.75
class                      Third
deck                     unknown
embark_town           Queenstown
alone                          y
Name: 10, dtype: object
1
0.62118226
